**INSTRUCTION PROMPTING**

This file contains code to generate literal translations using Instruction prompting.

You will need to add your OpenAI API key to get the results of prompting. For classsification, the finetuned model is linked to our account. Please ask for the OpenAI API to run the classification if needed.


In [ ]:
!pip install openai
!pip install datasets
!pip install git+https://github.com/google-research/bleurt.git
!pip install transformers
!pip install bert_score
!git clone https://github.com/google-research/bleurt.git
%cd bleurt
!pip install .
%cd /content/
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip .
!unzip BLEURT-20.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.3 MB/s eta 0:0

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import openai
import os
import pandas as pd
import time
import string
from datasets import load_metric, load
from bleurt import score as bleurt_score
from bert_score import score as bert_score

In [ ]:
import torch
import random
import numpy as np

torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Confirm that the GPU is detected
assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


In [ ]:
test_df = pd.read_csv('../Datasets/Prompting/prompting_test.csv')
test_df['Literal Translation'] = ''

In [ ]:
classification_key = "<replace_with_your_key>"
prompting_key = "<ask_for_our_model_key>"
openai.api_key = prompting_key

In [ ]:
def get_pred(res, labels):
  text = res["choices"][0]["text"].translate(str.maketrans('', '', string.punctuation))
  pred_tokens = text.split(" ")
  pred = "Metaphor"
  for i in pred_tokens:
    if i in labels:
      pred = i
      break
  time.sleep(1)
  return pred.replace("'", "")

In [ ]:
def get_type(text):
  openai.api_key = classification_key
  res = openai.Completion.create(model='ada:ft-personal-2023-05-15-01-40-58', prompt=text, temperature=0, top_p=1.0, max_tokens = 100)
  return get_pred(res, ["Sarcasm","Simile","Metaphor","Idiom"])

In [ ]:
# call completion model for instruction prompting
def instruction_prompting(df):
  from datasets.tasks import text_classification
  for ind in df.index:
    text = df['Hypothesis'][ind]
    fig_type= get_type(text)
    prompt = "For the given figurative sentence of type " + fig_type + ", provide a literal translation: " + text + "\n"

    print(prompt, end = "")
    lit_translation = openai.Completion.create(
      # model="text-curie-001",
      model="text-davinci-003",
      prompt=prompt, 
      max_tokens = 3000
    )
    print(lit_translation.choices[0].text.strip('\n\n') + '\n\n')
    df.loc[ind, ['Literal Translation']] = lit_translation.choices[0].text.strip('\n\n')
    time.sleep(1)

  print("DONE!")
  return df

In [ ]:
test_df = instruction_prompting(test_df)
test_df.shape

For the given figurative sentence of type Sarcasm, provide a literal translation: I heard a knock on my door at 2 am in the morning, and was super excited because I didn't know who it was.
I heard a knock on my door at 2 am in the morning, and was not pleased because I did not know who it was.


For the given figurative sentence of type Sarcasm, provide a literal translation: I was very proud of my sense of judgement when I found out that the person who I admired for their great vision was actually really mean to people
I was very embarrassed by my sense of judgement when I found out that the person who I admired for their great vision was actually really unkind to people.


For the given figurative sentence of type Sarcasm, provide a literal translation: I adore it when people blast their music during the week at 3am and disrupt my sleep
I do not appreciate it when people play their music loudly during the week at 3am and disturb my sleep.


For the given figurative sentence of type S

(400, 10)

In [ ]:
test_df

,Unnamed: 0,Index,Hypothesis,Premise,Type,Explanation,Label,SarcasmSimilarity,Source,Literal Translation
0,331,50,I heard a knock on my door at 2 am in the morn...,I heard a knock on my door at 2 am in the morn...,Sarcasm,Unknown people knocking on someone's door at 2...,Contradiction,UNQ,FLUTE,I heard a knock on my door at 2 am in the morn...
1,233,1630,I was very proud of my sense of judgement when...,I was seriously doubting my sense of judgement...,Sarcasm,: Someone who is admired for their great visio...,Contradiction,UNQ,FLUTE,I was very embarrassed by my sense of judgemen...
2,66,2112,I adore it when people blast their music durin...,I despise it when people blast their music dur...,Sarcasm,People blasting their music during the wee hou...,Contradiction,UNQ,FLUTE,I do not appreciate it when people play their ...
3,374,1833,I am pleased because the last thing I need is ...,I've been getting really bad headaches recently,Sarcasm,Having a headache makes someone's day worse as...,Contradiction,UNQ,FLUTE,I am not pleased because I need a headache to ...
4,452,1057,I was extremely happy to find out that all my ...,I was extremely broken to find out that all my...,Sarcasm,Studying for an annual exam is often a waste o...,Contradiction,UNQ,FLUTE,I was extremely disappointed to find out that ...
...,...,...,...,...,...,...,...,...,...,...
395,332,3260,I know that woman too well to ever want to lay...,I know that woman too well to ever want to hur...,Idiom,To lay a finger on someone usually refers to h...,Entailment,UNQ,FLUTE,"I understand that woman very well, so there is..."
396,527,7919,‘ That spoke volumes for the character as well...,‘ That provided substantial information for th...,Idiom,NaN,Entailment,NaN,IMPLI,The devotion and dedication of our squad was v...
397,569,6303,It 's all water under the bridge ; you do n't ...,It 's all in the past ; you do n't want to hea...,Idiom,NaN,Entailment,NaN,IMPLI,I apologize if you feel I have been unfairly t...
398,222,6481,The road and railway from Mallaig go south han...,The road and railway from Mallaig go south in ...,Idiom,NaN,Entailment,NaN,IMPLI,The road and railway from Mallaig travel toget...


In [ ]:
def calculate_bleurt(df):
  # BLEURT calculation
  scorer = bleurt_score.BleurtScorer('/content/BLEURT-20')
  bleurt_scores = scorer.score(references=df['Premise'], candidates=df['Literal Translation'])
  print(bleurt_scores)
  return bleurt_scores

def calculate_bertscore(df):
  # BERTScore calculation
  bertscore = load_metric('bertscore')
  bert_scores = bertscore.compute(predictions=df['Literal Translation'], references=df['Premise'], lang="en")
  print(bert_scores['f1'])
  return bert_scores['f1']

def calculate_littransscore(bert_scores, bleurt_scores):
  lit_trans_scores = []
  for ind in range(len(bleurt_scores)):
    lit_trans_scores.append((bert_scores[ind] + bleurt_scores[ind]) * 50.0)
  return lit_trans_scores

In [ ]:
bleurt_scores = calculate_bleurt(test_df)

[0.651700496673584, 0.7913245558738708, 0.7951133847236633, 0.3949957489967346, 0.8219431638717651, 0.8425740599632263, 0.7764391303062439, 0.8090178370475769, 0.6074761152267456, 0.7335677146911621, 0.8328306078910828, 0.8227601647377014, 0.6258907914161682, 0.8417148590087891, 0.4605565071105957, 0.6876253485679626, 0.8275336623191833, 0.6876623630523682, 0.6057666540145874, 0.6163531541824341, 0.7417724132537842, 0.8425712585449219, 0.7427592873573303, 0.7442743182182312, 0.5762766599655151, 0.7486098408699036, 0.661898672580719, 0.6602763533592224, 0.7916660904884338, 0.6112224459648132, 0.7783849239349365, 0.7806166410446167, 0.6694930195808411, 0.7649232149124146, 0.8139833807945251, 0.7557203769683838, 0.8867239952087402, 0.6118161678314209, 0.8059205412864685, 0.652567446231842, 0.7431455850601196, 0.6050986647605896, 0.6360496282577515, 0.8663012385368347, 0.7115785479545593, 0.8732346296310425, 0.5799435973167419, 0.7401446104049683, 0.6272661089897156, 0.6354820728302002, 0.

In [ ]:
bert_scores = calculate_bertscore(test_df)

<ipython-input-13-96d02919035d>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bertscore = load_metric('bertscore')


[0.955495297908783, 0.9632682800292969, 0.9784218668937683, 0.8691365718841553, 0.9703025221824646, 0.9720851182937622, 0.9541335105895996, 0.9288199543952942, 0.9103638529777527, 0.9527586698532104, 0.9727256894111633, 0.9425451755523682, 0.9129899740219116, 0.9750657677650452, 0.9127305746078491, 0.9242489337921143, 0.988666832447052, 0.9185469746589661, 0.9175359606742859, 0.9201350212097168, 0.9635557532310486, 0.9772569537162781, 0.9743024706840515, 0.9561211466789246, 0.9172054529190063, 0.9528027176856995, 0.9563795328140259, 0.9169966578483582, 0.9613847732543945, 0.91166090965271, 0.9856020212173462, 0.9739524126052856, 0.9598665237426758, 0.9725601673126221, 0.9795530438423157, 0.9733244776725769, 0.9951736330986023, 0.9198363423347473, 0.9602431654930115, 0.9424676299095154, 0.9662973284721375, 0.9299784302711487, 0.9394963383674622, 0.9918731451034546, 0.9642729163169861, 0.9892102479934692, 0.9314169883728027, 0.9509730339050293, 0.9530787467956543, 0.9367009997367859, 0.9

In [ ]:
print("Range of BLEURT: ")
print(min(bleurt_scores))
print(max(bleurt_scores), end='\n')

print("Range of BERTScore: ")
print(min(bert_scores))
print(max(bert_scores), end='\n')

Range of BLEURT: 
0.2537620961666107
0.9615190029144287
Range of BERTScore: 
0.81241774559021
1.0000001192092896


In [ ]:
# Compute literal translation score
lit_trans_scores = calculate_littransscore(bert_scores, bleurt_scores)
test_df['Score'] = lit_trans_scores
test_df['BLEURT'] = bleurt_scores
test_df['BERT'] = bert_scores

lit_trans_scores_scores_metaphor = test_df.groupby('Type').get_group('Metaphor')['Score']
lit_trans_scores_scores_idiom = test_df.groupby('Type').get_group('Idiom')['Score']
lit_trans_scores_scores_simile = test_df.groupby('Type').get_group('Simile')['Score']
lit_trans_scores_scores_sarcasm = test_df.groupby('Type').get_group('Sarcasm')['Score']

print(min(lit_trans_scores))
print(max(lit_trans_scores))

print('Average literal translation score for metaphor: %d' %(sum(lit_trans_scores_scores_metaphor)/len(lit_trans_scores_scores_metaphor)))
print('Average literal translation score for idiom: %d' %(sum(lit_trans_scores_scores_idiom)/len(lit_trans_scores_scores_idiom)))
print('Average literal translation score for simile: %d' %(sum(lit_trans_scores_scores_simile)/len(lit_trans_scores_scores_simile)))
print('Average literal translation score for sarcasm: %d' %(sum(lit_trans_scores_scores_sarcasm)/len(lit_trans_scores_scores_sarcasm)))


54.781682789325714
98.07595610618591
Average literal translation score for metaphor: 79
Average literal translation score for idiom: 80
Average literal translation score for simile: 72
Average literal translation score for sarcasm: 83


In [ ]:
# Export results
test_zero_shot = test_df.loc[:,['Hypothesis','Premise','Literal Translation','Score', 'BLEURT', 'BERT', 'Type','Source']]
test_zero_shot.to_csv('../Results/Prompting/instruction_tuning_new.csv')